### Calculate Basic User Stats
Given a single user name and ID, we want to calculate all the basica stats and save it back to the DB. 
- Average CS per game

In [ ]:
%config IPCompleter.greedy=True
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.driver.extraClassPath', './lib/postgresql-42.2.5.jar').getOrCreate()
url = 'jdbc:postgresql://postgres:5432/lol'
properties = {'user': 'lol', 'password': 'lol'}

In [ ]:
summoner_df = spark.read.jdbc(url=url, table="(select * from match limit 1) s", properties=properties)
summoner_df.show()

In [ ]:
summoner_name = 'name_here'

#### Getting all matches for this summoner

In [ ]:
get_all_matches_query = f"(select m.* from summoner s join matches map on s.id = map.summoner_id join match m on map.match_id = m.id where s.name = '{summoner_name}') a"
matches_df = spark.read.jdbc(url=url, table=get_all_matches_query, properties=properties)
matches_df.show()

Converting `info` column with json type to dataframe

In [ ]:
from pyspark.sql import SQLContext
sqlc = SQLContext(spark.sparkContext)
match_df = matches_df.first()
info_df = sqlc.read.json(matches_df.rdd.map(lambda r: r.info))
info_df.printSchema()

In [ ]:
# find out the participantId
# pull participants->stats->goldEarned
total_gold = 0
total_match = 0
total_average_gold_per_minute = 0
for info in info_df.collect():
    game_minutes = info.gameDuration/60
    participant_id = 0
    for participant in info.participantIdentities:
        if participant.player.summonerName == summoner_name:
            participant_id = participant.participantId
    # found participantId
    for p in info.participants: 
        if participant_id == p.participantId:
            total_gold += p.stats.goldEarned
            total_match += 1
            average_gold_per_minute = p.stats.goldEarned / game_minutes
            print("average_gold_per_minute: ", average_gold_per_minute)
            total_average_gold_per_minute += average_gold_per_minute
print("summoner: ", summoner_name)
print("total_gold: ", total_gold)
print("total_match: ", total_match)
print("average_gold_per_match: ", total_gold/total_match)
print("average_gold_per_minute: ", total_average_gold_per_minute / total_match)
            

In [ ]:
from pyspark.sql import Row
from collections import OrderedDict
from pyspark.sql.functions import col, avg


def convert_to_row(d: dict) -> Row:
    return Row(**OrderedDict(sorted(d.items())))

list = []
for info in info_df.collect():
    game_stat = {}
    game_stat['gameDuration'] = info.gameDuration/60
    participant_id = 0

    for participant in info.participantIdentities:
        if participant.player.summonerName == summoner_name:
            participant_id = participant.participantId
    # found participantId
    for p in info.participants: 
        if participant_id == p.participantId:
            game_stat['goldEarned'] = p.stats.goldEarned
            game_stat['visionScore'] = p.stats.visionScore
            game_stat['totalDamageDealt'] = p.stats.totalDamageDealt
            game_stat['totalDamageTaken'] = p.stats.totalDamageTaken
            game_stat['kills'] = p.stats.kills
            game_stat['deaths'] = p.stats.deaths
            game_stat['assists'] = p.stats.assists
            game_stat['goldPerMinDeltas_10'] = p.timeline.goldPerMinDeltas.__getitem__("0-10") if p.timeline.goldPerMinDeltas else 0            
            game_stat['goldPerMinDeltas_20'] = p.timeline.goldPerMinDeltas.__getitem__('10-20') if p.timeline.goldPerMinDeltas else 0            
            game_stat['goldPerMinDeltas_30'] = p.timeline.goldPerMinDeltas.__getitem__('20-30') if p.timeline.goldPerMinDeltas else 0
            game_stat['creepsPerMinDeltas_10'] = p.timeline.creepsPerMinDeltas.__getitem__('0-10') if p.timeline.goldPerMinDeltas else 0
            game_stat['creepsPerMinDeltas_20'] = p.timeline.creepsPerMinDeltas.__getitem__('10-20') if p.timeline.goldPerMinDeltas else 0
            game_stat['creepsPerMinDeltas_30'] = p.timeline.creepsPerMinDeltas.__getitem__('20-30') if p.timeline.goldPerMinDeltas else 0
            list.append(game_stat)

df = spark.sparkContext.parallelize(list).map(convert_to_row).toDF()           
new_df = df.select(avg("goldEarned"), avg("visionScore"), avg("totalDamageDealt"), 
                   avg("totalDamageTaken"), avg("kills"), avg("deaths"), avg("assists"),
                  avg("goldPerMinDeltas_10"), avg("goldPerMinDeltas_20"), avg("goldPerMinDeltas_30"),
                  avg("creepsPerMinDeltas_10"), avg("creepsPerMinDeltas_20"), avg("creepsPerMinDeltas_30"))
new_df.show()
            

In [ ]:
df.createOrReplaceTempView("basic_stats")
spark.sql("select * from basic_stats")